# Detection of faces and cropping using predefined model

In [2]:
import os
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np

import dlib
import pickle

from collections import defaultdict

In [8]:
folder_path = r"G:\my-proj\colour-analysis-profession\mixed_photos\Light" 

In [9]:
image_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif'))]
len(image_files)

30

In [10]:
# Initialize dlib's CNN-based face detector
cnn_detector = dlib.cnn_face_detection_model_v1("mmod_human_face_detector.dat")

In [11]:
def extract_face_regions(image_path):
    """
    Extracts the face regions from the detected faces.
    """
    # Check if the file exists
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image file not found: {image_path}")

    # Read the image
    image = cv.imread(image_path)
    if image is None:
        raise ValueError(f"Failed to load the image. Check the file format or path: {image_path}")
    
    # Convert to RGB
    rgb_image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    
    # Detect faces using the CNN detector (assuming cnn_detector is defined)
    faces_cnn = cnn_detector(rgb_image)
    
    # Extract face regions
    face_regions = []
    for face in faces_cnn:
        x, y, w, h = (face.rect.left(), face.rect.top(), face.rect.width(), face.rect.height())
        face_crop = image[y:y+h, x:x+w]  # Crop the face region
        face_regions.append(face_crop)
    
    return face_regions


In [12]:
%%time
cropped_faces=[]
for i in image_files:
    cropped_faces.extend(extract_face_regions(i))

CPU times: total: 1min 31s
Wall time: 2min 3s


In [13]:
# Directory to save cropped faces
output_dir = "l1"
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist

# Save each cropped face as an image
saved_count = 0
for idx, face in enumerate(cropped_faces):
    if face is None or face.size == 0:  # Check if the face is empty
        print(f"Skipping empty or invalid cropped face at index {idx}")
        continue

    # Construct the output file path
    output_path = os.path.join(output_dir, f"face_{idx + 1}.jpg")
    
    # Save the image using OpenCV
    cv.imwrite(output_path, face)
    saved_count += 1

print(f"Saved {saved_count} valid cropped faces to the folder: {output_dir}")

Skipping empty or invalid cropped face at index 15
Saved 34 valid cropped faces to the folder: l1


In [1]:
import streamlit as st
import torch
import torch.nn as nn
from torchvision import transforms
from torchvision.models import resnet18
from PIL import Image
import cv2 as cv
import dlib
import os

# Load pre-trained ResNet model for skin tone prediction
model = resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 3)  # 3 classes: Dark, Light, Medium
model.load_state_dict(torch.load("resnet_model.pth"))
model.eval()

# Define the same transformations used during training
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the dlib CNN face detector
cnn_detector = dlib.cnn_face_detection_model_v1("mmod_human_face_detector.dat")

# Extended dataset with color names and hex codes
data = {
    "Skin Color Type": ["Light", "Medium", "Dark"],
    "Colour Pattern": [
        "Bright and pastel shades",
        "Warm and earthy tones",
        "Rich and bold colors"
    ],
    "Single Colour": [
        [("Lavender", "#E6E6FA"), ("Mint", "#98FF98"), ("Sky Blue", "#87CEEB"), ("Peach", "#FFDAB9"), ("Powder Pink", "#FFC0CB"), ("Soft Yellow", "#FFFACD"), ("Ivory", "#FFFFF0")],
        [("Olive", "#808000"), ("Coral", "#FF7F50"), ("Beige", "#F5F5DC"), ("Burnt Orange", "#CC5500"), ("Mustard Yellow", "#FFDB58"), ("Turquoise", "#40E0D0"), ("Cinnamon Brown", "#7B3F00")],
        [("Maroon", "#800000"), ("Navy Blue", "#000080"), ("Emerald Green", "#50C878"), ("Gold", "#FFD700"), ("Deep Purple", "#673AB7"), ("Ruby Red", "#9B111E"), ("Charcoal Gray", "#36454F")]
    ],
    "Dual Colour": [
        [("Lavender + Mint", ["#E6E6FA", "#98FF98"]), ("Sky Blue + Peach", ["#87CEEB", "#FFDAB9"]), ("Soft Yellow + Powder Pink", ["#FFFACD", "#FFC0CB"])],
        [("Olive + Coral", ["#808000", "#FF7F50"]), ("Beige + Burnt Orange", ["#F5F5DC", "#CC5500"]), ("Turquoise + Mustard Yellow", ["#40E0D0", "#FFDB58"])],
        [("Maroon + Navy Blue", ["#800000", "#000080"]), ("Emerald Green + Gold", ["#50C878", "#FFD700"]), ("Deep Purple + Ruby Red", ["#673AB7", "#9B111E"])]
    ],
    "Multicolour": [
        [("Lavender + Mint + Sky Blue", ["#E6E6FA", "#98FF98", "#87CEEB"])],
        [("Olive + Coral + Beige", ["#808000", "#FF7F50", "#F5F5DC"])],
        [("Maroon + Navy Blue + Emerald Green", ["#800000", "#000080", "#50C878"])]
    ]
}

# Function to detect faces and extract face regions
def extract_face_regions(image_path):
    # Read the image
    image = cv.imread(image_path)
    if image is None:
        raise ValueError(f"Failed to load the image. Check the file format or path: {image_path}")
    
    # Convert to RGB
    rgb_image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    
    # Detect faces
    faces_cnn = cnn_detector(rgb_image)
    
    # Extract face regions
    face_regions = []
    for face in faces_cnn:
        x, y, w, h = (face.rect.left(), face.rect.top(), face.rect.width(), face.rect.height())
        face_crop = image[y:y+h, x:x+w]
        face_regions.append(face_crop)
    
    return face_regions

# Function to predict skin tone for a given face region
def predict_skin_tone(face_region):
    # Convert the face region to PIL format
    face_image = Image.fromarray(cv.cvtColor(face_region, cv.COLOR_BGR2RGB))
    
    # Preprocess the image
    image_tensor = transform(face_image).unsqueeze(0)
    
    # Predict skin tone
    with torch.no_grad():
        outputs = model(image_tensor)
        _, predicted = torch.max(outputs, 1)
    
    class_names = ['Dark', 'Light', 'Medium']  # Match folder names
    return class_names[predicted.item()]

# Function to render colors as blocks
def render_colors(color_data):
    for name, colors in color_data:
        if isinstance(colors, list):  # For dual or multicolor palettes
            st.write(f"**{name}:**")
            cols = st.columns(len(colors))
            for idx, color in enumerate(colors):
                cols[idx].markdown(f'<div style="width: 50px; height: 25px; background-color: {color};"></div>', unsafe_allow_html=True)
        else:  # For single colors
            st.write(f"**{name}:**")
            st.markdown(f'<div style="width: 50px; height: 25px; background-color: {colors};"></div>', unsafe_allow_html=True)

# Streamlit application
st.title("Skin Tone Color Analysis")

# Upload an image
uploaded_image = st.file_uploader("Upload an image to analyze skin tone:", type=["jpg", "jpeg", "png"])

if uploaded_image:
    # Save uploaded image
    st.image(uploaded_image, caption="Uploaded Image", use_column_width=True)
    with open("uploaded_image.jpg", "wb") as f:
        f.write(uploaded_image.getbuffer())
    
    # Detect face regions
    face_regions = extract_face_regions("uploaded_image.jpg")
    if not face_regions:
        st.error("No faces detected in the uploaded image.")
    else:
        # Predict skin tone for the first detected face
        skin_tone = predict_skin_tone(face_regions[0])
        st.write(f"**Predicted Skin Tone:** {skin_tone}")
        
        # Display best colors for the predicted skin tone
        index = data["Skin Color Type"].index(skin_tone)
        
        st.write(f"### Best Colors for {skin_tone} Skin Tone")
        st.write(f"**Colour Pattern:** {data['Colour Pattern'][index]}")
        
        st.write("### Single Colours")
        render_colors(data["Single Colour"][index])
        
        st.write("### Dual Colours")
        render_colors(data["Dual Colour"][index])
        
        st.write("### Multicolours")
        render_colors(data["Multicolour"][index])


C:\Users\lenovo\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\lenovo\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\lenovo\AppData\Local\Temp\ipykernel_23244\2505560897.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be fl

In [2]:
predicted

NameError: name 'predicted' is not defined